<a href="https://colab.research.google.com/github/Macia-Comella/03MAIR---Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Maci%C3%A0_Comella_Barb%C3%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Macià Comella Barbé  <br>
Url: https://github.com/Macia-Comella/03MAIR---Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Maci%C3%A0_Comella_Barb%C3%A9.ipynb <br>
Google Colab: https://colab.research.google.com/drive/1FGZOnWffgCmwZOaOAiDOFlKo6aX0BwOK?usp=sharing <br>
Problema:
>1. Sesiones de doblaje <br>

Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible. Los datos son:

Número de actores: 10  
Número de tomas : 30  
Actores/Tomas : https://bit.ly/36D8IuK

....







                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

### Respuesta

**Espacio de soluciones:**  
El espacio de soluciones se representa con lista de listas que contiene las sesiones programadas cada día. Cada lista representa un día y este contiene las sesiones de dicho día. El tamaño de las listas deberá limitarse a 6 ya que no se pueden programar más de 6 tomas por día.

Asociada a esta lista de programación de tomas se generará una lista de listas con los actores que deben desplazarse al estudio cada día siguiendo la misma lógica (cada lista representa un día y contiene los actores necesarios para tal día)

In [1]:
import random
import numpy as np

In [101]:
# Carga datos actores
planificacion = {'1': [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                 '2': [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
                 '3': [0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
                 '4': [1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
                 '5': [0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
                 '6': [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
                 '7': [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
                 '8': [1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
                 '9': [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
                 '10': [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
                 '11': [1, 1, 1, 0, 1, 0, 0, 1, 0, 0],
                 '12': [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
                 '13': [1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
                 '14': [1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                 '15': [1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
                 '16': [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
                 '17': [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                 '18': [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                 '19': [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                 '20': [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
                 '21': [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
                 '22': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                 '23': [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                 '24': [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                 '25': [1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
                 '26': [1, 0, 1, 0, 1, 0, 0, 0, 1, 0],
                 '27': [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
                 '28': [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                 '29': [1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
                 '30': [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]}

for k in planificacion:
  lista = []
  for numero, actor in enumerate(planificacion[k]):
    if actor == 1:
      lista.append(numero+1)
  planificacion[k] = lista

{'1': [1, 2, 3, 4, 5], '2': [3, 4, 5], '3': [2, 5, 7], '4': [1, 2, 7, 8], '5': [2, 4, 8], '6': [1, 2, 4, 5], '7': [1, 2, 4, 5], '8': [1, 2, 6], '9': [1, 2, 4], '10': [1, 2, 6, 9], '11': [1, 2, 3, 5, 8], '12': [1, 2, 3, 4, 6], '13': [1, 4, 5], '14': [1, 3, 6], '15': [1, 2, 7], '16': [4, 10], '17': [1, 3], '18': [3, 6], '19': [1, 3], '20': [1, 3, 4, 5], '21': [6, 8], '22': [1, 2, 3, 4], '23': [1, 3], '24': [3, 6], '25': [1, 2, 4, 10], '26': [1, 3, 5, 9], '27': [4, 5], '28': [1, 4], '29': [1, 5, 6], '30': [1, 4]}


In [64]:
# Espacio de soluciones

def genera_programacion_aleatoria():
  programacion = [[]]
  tomas_a_programar = list(range(1,31))

  while len(tomas_a_programar) > 0:
    for dia in programacion:
      programado = False
      if len(dia) < 6:
        toma_programada = random.choice(tomas_a_programar)
        dia.append(toma_programada)
        tomas_a_programar.remove(toma_programada)
        programado = True

    if not programado:
      programacion.append([])
      programado = False

  return programacion

In [119]:
def programa_actores(programacion, plan):
  programacion_actores = []
  for dia, tomas_dia in enumerate(programacion):
    programacion_actores.append([])
    for toma in tomas_dia:
      for tomas_plan in plan[str(toma)]:
        programacion_actores[dia].append(tomas_plan)

  for dia in range(len(programacion_actores)):
    programacion_actores[dia] = set(programacion_actores[dia])

  return programacion_actores


In [120]:
programacion = genera_programacion_aleatoria()
print(programacion)
print(programa_actores(programacion, planificacion))

[[19, 20, 26, 8, 9, 18], [24, 17, 22, 1, 28, 12], [5, 30, 13, 6, 3, 16], [29, 7, 27, 4, 14, 11], [25, 15, 10, 2, 21, 23]]
[{1, 2, 3, 4, 5, 6, 9}, {1, 2, 3, 4, 5, 6}, {1, 2, 4, 5, 7, 8, 10}, {1, 2, 3, 4, 5, 6, 7, 8}, {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}]


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

In [ ]:
#Respuesta


#Diseño
- ¿Que técnica utilizo? ¿Por qué?

In [ ]:
#Respuesta